# Portfolio Metrics

It's smart to try and calculate portfolio metrics.


With the sharpe ratio, where is one place to check the sectors? How many sectors are there that you can get the maximum diversification benefit out of?

For example, I am invested in electric vehicles, bitcoin, grocery chains, banks, and financial tech.

What are some useful things for me to track in my own individual investments from a theoretical perspective?



In [46]:
metrics = pd.DataFrame({"Name": 
    ['Net Portfolio Gains'], 
    "Number": [sum(df['Total Return'])]})

In [76]:
# Calculating sharpe ratio of my portfolio


In [47]:
# Calculating Beta
goog = pdr.get_data_yahoo("JMIA", start = "2021-01-01", end="2021-3-30")
spy = pdr.get_data_yahoo("SPY", start = "2021-01-01", end="2021-3-30")
tsla = pdr.get_data_yahoo("TSLA", start = "2021-01-01", end="2021-3-30")
pltr = pdr.get_data_yahoo("PLTR", start = "2021-01-01", end="2021-3-30")


In [48]:
plt.style.use('ggplot')

In [49]:
get_data("JMIA")

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-12-31,42.599998,39.835999,42.500000,40.349998,8581900,40.349998
2021-01-04,40.570000,35.520000,39.939999,35.680000,11407900,35.680000
2021-01-05,37.439999,34.110001,34.349998,36.240002,7360100,36.240002
2021-01-06,36.689999,33.000000,35.799999,33.209999,10494000,33.209999
2021-01-07,39.353001,34.590000,34.820000,37.849998,13493800,37.849998
...,...,...,...,...,...,...
2021-04-28,32.700001,30.799999,31.750000,32.310001,2833000,32.310001
2021-04-29,32.910000,29.660000,32.509998,30.410000,4630200,30.410000
2021-04-30,31.850000,29.715000,29.990000,30.420000,4308000,30.420000


In [50]:
tickers = ["JMIA", "SPY", "TSLA", "PLTR"]
tickers = df.Symbol.tolist()

def pct(tickers):
    data = {}
    deviations = {}
    for tick in tickers:
        data[tick] = pdr.get_data_yahoo(tick, start = 
            "2021-01-01", end="2021-3-30").Close.pct_change()[1:]
    # Visualizing
    plt.figure(figsize=(10,8))
    for k,v in data.items():
        v.plot()
        deviations[k] = np.std(v.values)
        
    plt.legend(labels = tickers)
    plt.show()
    
    return deviations

In [51]:
dev = pct(tickers)

KeyError: 'Date'

In [ ]:
sorted_tuples = sorted(dev.items(), key=lambda item: item[1])
sorted_dev = {k: v for k, v in sorted_tuples}

plt.figure(figsize=(10,8))
sns.barplot(x=list(sorted_dev.keys()), y=list(sorted_dev.values()))
plt.xticks(rotation=90)
plt.title("Standard Deviations");

In [ ]:
metrics

# Sharpe Ratio Markowitz

https://towardsdatascience.com/python-markowitz-optimization-b5e1623060f5

In [ ]:
# Containing price data for my stocks
stocks = pdr.get_data_yahoo("PLTR", start = "2021-01-01", 
        end="2021-3-30").Close
tickers = ["TSLA", "JMIA","MARA","PLTR"] # df.Symbol.tolist()
tickers.remove("PLTR")
for t in tickers:
    close_prices = pdr.get_data_yahoo(t, start = "2021-01-01", 
        end="2021-3-30").Close
    stocks = pd.concat([stocks, close_prices], axis = 1)
stocks.columns = ["PLTR"] + tickers
stocks.head()

In [ ]:
# Normalizing the data
log_ret = np.log(stocks.shift(1))
log_ret.head()

In [ ]:
np.random.seed(42)
num_ports = 6000
all_weights = np.zeros((num_ports, len(stocks.columns)))
ret_arr = np.zeros(num_ports)
vol_arr = np.zeros(num_ports)
sharpe_arr = np.zeros(num_ports)

for x in range(num_ports):
    # Weights
    weights = np.array(np.random.random(4))
    weights = weights/np.sum(weights)
    
    # Save weights
    all_weights[x,:] = weights
    
    # Expected return
    ret_arr[x] = np.sum( (log_ret.mean() * weights * 252))
    
    # Expected volatility
    vol_arr[x] = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov()*252, weights)))
    
    # Sharpe Ratio
    sharpe_arr[x] = ret_arr[x]/vol_arr[x]

In [ ]:
print("Max sharpe ratio in the array: {}".format(sharpe_arr.max()))
print("It's location in the array: {}".format(sharpe_arr.argmax()))

In [ ]:
print(all_weights[1893,:])

max_sr_ret = ret_arr[sharpe_arr.argmax()]
max_sr_vol = vol_arr[sharpe_arr.argmax()]

Find the portfolio that the maximum sharpe ratio. Once you have that, you know that the efficient thing for me to do is to take a position on this one and on the risk free asset. Potentially a leveraged position. The MVE is my portfolio of risky asset.

Think - I will take a __ position in the MVE, and the rest on the risk free asset.

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(vol_arr, ret_arr, c=sharpe_arr, cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.scatter(max_sr_vol, max_sr_ret,c='red', s=50) # red dot
plt.show()

In [80]:
def get_ret_vol_sr(weights):
    weights = np.array(weights)
    ret = np.sum(log_ret.mean() * weights) * 252
    vol = np.sqrt(np.dot(weights.T, np.dot(log_ret.cov()*252, weights)))
    sr = ret/vol
    return np.array([ret, vol, sr])

def neg_sharpe(weights):
# the number 2 is the sharpe ratio index from the get_ret_vol_sr
    return get_ret_vol_sr(weights)[2] * -1

def check_sum(weights):
    #return 0 if sum of the weights is 1
    return np.sum(weights)-1

In [81]:
cons = ({'type':'eq','fun':check_sum})
bounds=((0,1),(0,1),(0,1),(0,1))
init_guess=[0.25]*4

In [84]:
opt_results = minimize(neg_sharpe, init_guess, method='SLSQP',bounds=bounds,
                      constraints=cons)

NameError: name 'minimize' is not defined

# Market Overview

[Getting Live Data](https://towardsdatascience.com/python-how-to-get-live-market-data-less-than-0-1-second-lag-c85ee280ed93)

In [83]:
candle("NIO")

[*********************100%***********************]  1 of 1 completed


In [82]:
candle("JMIA")

[*********************100%***********************]  1 of 1 completed


In [59]:
figs = [candle(t) for t in df.Symbol.tolist()]
    

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [56]:
from plotly.subplots import make_subplots

In [66]:
fig = make_subplots(rows=1, cols=2)

data = yf.download(tickers="JMIA", period="5d", interval="5m")

#Candlestick
fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'], name = 'market data'), row = 1, col = 1)

data = yf.download(tickers="TSLA", period="5d", interval="5m")

fig.add_trace(go.Candlestick(x=data.index,
                open=data['Open'],
                high=data['High'],
                low=data['Low'],
                close=data['Close'], name = 'market data'), row = 1, col = 2)

fig.update_layout(height=600, width=800, title_text="Side By Side Subplots")
fig.show()


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Funds take out way too much margin. They panic sell.

In [44]:
oversold = ["VIAC", "DISCA", 'IDEX'] # 20-25% oversold 
# Set trailing stop loss after $60 for discovery
NFT = ['CSCW']
wait_for_dip = ['CRWD']
market = ['QQQ','VOO','^VIX']

charging_stations = ['CHPT', 'BLNK']